In [7]:
import pandas as pd
import numpy as np

df = pd.read_csv('data/nyc-east-river-bicycle-counts.csv', index_col=0)

df.head()

#In order to represent precipitation as numeric, convert S values to a separate boolean column 'Snowed', and bin T (Trace)
#with 0.01
df['Snowed'] = df.Precipitation.map(lambda x: 'S' in x)
df.Precipitation.replace(regex=r"\(S\)", value='', inplace=True)
df.Precipitation.replace('T', '0.01', inplace=True)
df.Precipitation = df.Precipitation.astype('float')

#Make Date datetime, and Day weekday names
df.Date = pd.to_datetime(df.Date)
df.Day = pd.to_datetime(df.Day).dt.day_name()

#Remove Total column:
df.drop('Total', axis=1, inplace=True)

df.head()

,Date,Day,High Temp (°F),Low Temp (°F),Precipitation,Brooklyn Bridge,Manhattan Bridge,Williamsburg Bridge,Queensboro Bridge,Snowed
0,2016-04-01,Friday,78.1,66.0,0.01,1704.0,3126,4115.0,2552.0,False
1,2016-04-02,Saturday,55.0,48.9,0.15,827.0,1646,2565.0,1884.0,False
2,2016-04-03,Sunday,39.9,34.0,0.09,526.0,1232,1695.0,1306.0,False
3,2016-04-04,Monday,44.1,33.1,0.47,521.0,1067,1440.0,1307.0,True
4,2016-04-05,Tuesday,42.1,26.1,0.00,1416.0,2617,3081.0,2357.0,False


In [43]:
#Reshape the data to a "long" format, to demonstrate pivot table operations with the Bridges as a multi-index

bridges = ['Brooklyn Bridge', 'Manhattan Bridge', 'Williamsburg Bridge', 'Queensboro Bridge']
value_cols = ['Date', 'Day', 'High Temp (°F)', 'Low Temp (°F)', 'Precipitation', 'Snowed']
bikes = df.melt(id_vars=value_cols, value_vars=bridges,
            var_name='Bridge', value_name='Crossings')
bikes.head()

,Date,Day,High Temp (°F),Low Temp (°F),Precipitation,Snowed,Bridge,Crossings
0,2016-04-01,Friday,78.1,66.0,0.01,False,Brooklyn Bridge,1704.0
1,2016-04-02,Saturday,55.0,48.9,0.15,False,Brooklyn Bridge,827.0
2,2016-04-03,Sunday,39.9,34.0,0.09,False,Brooklyn Bridge,526.0
3,2016-04-04,Monday,44.1,33.1,0.47,True,Brooklyn Bridge,521.0
4,2016-04-05,Tuesday,42.1,26.1,0.00,False,Brooklyn Bridge,1416.0


In [9]:
df.head()

,Date,Day,High Temp (°F),Low Temp (°F),Precipitation,Brooklyn Bridge,Manhattan Bridge,Williamsburg Bridge,Queensboro Bridge,Snowed
0,2016-04-01,Friday,78.1,66.0,0.01,1704.0,3126,4115.0,2552.0,False
1,2016-04-02,Saturday,55.0,48.9,0.15,827.0,1646,2565.0,1884.0,False
2,2016-04-03,Sunday,39.9,34.0,0.09,526.0,1232,1695.0,1306.0,False
3,2016-04-04,Monday,44.1,33.1,0.47,521.0,1067,1440.0,1307.0,True
4,2016-04-05,Tuesday,42.1,26.1,0.00,1416.0,2617,3081.0,2357.0,False


In [46]:
first_pivot = bikes.pivot_table(index=['Date', 'Bridge'])
first_pivot.head(8)

Crossings  High Temp (°F)  Low Temp (°F)  \
Date       Bridge                                                          
2016-04-01 Brooklyn Bridge         1704.0            78.1           66.0   
           Manhattan Bridge        3126.0            78.1           66.0   
           Queensboro Bridge       2552.0            78.1           66.0   
           Williamsburg Bridge     4115.0            78.1           66.0   
2016-04-02 Brooklyn Bridge          827.0            55.0           48.9   
           Manhattan Bridge        1646.0            55.0           48.9   
           Queensboro Bridge       1884.0            55.0           48.9   
           Williamsburg Bridge     2565.0            55.0           48.9   

                                Precipitation  Snowed  
Date       Bridge                                      
2016-04-01 Brooklyn Bridge               0.01   False  
           Manhattan Bridge              0.01   False  
           Queensboro Bridge             0.01   False  
           Williamsburg Bridge           0.01   False  
2016-04-02 Brooklyn Bridge               0.15   False  
           Manhattan Bridge              0.15   False  
           Queensboro Bridge             0.15   False  
           Williamsburg Bridge           0.15   False

In [49]:
bikes.groupby(['Date', 'Bridge']).head()

,Date,Day,High Temp (°F),Low Temp (°F),Precipitation,Snowed,Bridge,Crossings
0,2016-04-01,Friday,78.1,66.0,0.01,False,Brooklyn Bridge,1704.0
1,2016-04-02,Saturday,55.0,48.9,0.15,False,Brooklyn Bridge,827.0
2,2016-04-03,Sunday,39.9,34.0,0.09,False,Brooklyn Bridge,526.0
3,2016-04-04,Monday,44.1,33.1,0.47,True,Brooklyn Bridge,521.0
4,2016-04-05,Tuesday,42.1,26.1,0.00,False,Brooklyn Bridge,1416.0
...,...,...,...,...,...,...,...,...
775,2016-04-26,Tuesday,60.1,46.9,0.24,False,Queensboro Bridge,2929.0
776,2016-04-27,Wednesday,62.1,46.9,0.00,False,Queensboro Bridge,4388.0
777,2016-04-28,Thursday,57.9,48.0,0.00,False,Queensboro Bridge,3657.0
778,2016-04-29,Friday,57.0,46.9,0.05,False,Queensboro Bridge,3348.0


In [52]:
bikes.pivot_table('Crossings', index='Day', columns='Bridge')

Bridge,Brooklyn Bridge,Manhattan Bridge,Queensboro Bridge,Williamsburg Bridge
Day,,,,
Friday,2361.20,4147.60,3373.60,5038.00
Monday,2465.00,4414.75,3504.25,5012.75
Saturday,1793.80,3037.40,2856.40,4045.40
Sunday,1979.50,3309.50,2905.00,4007.25
Thursday,2514.75,4623.50,3760.50,5518.50
Tuesday,1989.75,3816.50,3100.00,4559.00
Wednesday,2879.50,5226.00,4089.25,6016.75


In [57]:
bikes.groupby(['Day', 'Bridge'])['Crossings'].agg(np.mean).unstack()

Bridge,Brooklyn Bridge,Manhattan Bridge,Queensboro Bridge,Williamsburg Bridge
Day,,,,
Friday,2361.20,4147.60,3373.60,5038.00
Monday,2465.00,4414.75,3504.25,5012.75
Saturday,1793.80,3037.40,2856.40,4045.40
Sunday,1979.50,3309.50,2905.00,4007.25
Thursday,2514.75,4623.50,3760.50,5518.50
Tuesday,1989.75,3816.50,3100.00,4559.00
Wednesday,2879.50,5226.00,4089.25,6016.75


In [58]:
bikes.pivot_table('Crossings', index='Day', columns='Bridge',
                   aggfunc='sum', margins=True)

Bridge,Brooklyn Bridge,Manhattan Bridge,Queensboro Bridge,Williamsburg Bridge,All
Day,,,,,
Friday,82642.0,145166.0,118076.0,176330.0,522214.0
Monday,69020.0,123613.0,98119.0,140357.0,431109.0
Saturday,62783.0,106309.0,99974.0,141589.0,410655.0
Sunday,55426.0,92666.0,81340.0,112203.0,341635.0
Thursday,70413.0,129458.0,105294.0,154518.0,459683.0
Tuesday,55713.0,106862.0,86800.0,127652.0,377027.0
Wednesday,80626.0,146328.0,114499.0,168469.0,509922.0
All,476623.0,850402.0,704102.0,1021118.0,3052245.0


In [61]:
bikes.pivot_table(index='Day', columns='Bridge',
                   aggfunc={'Crossings':'sum',
                             'Precipitation':'mean'})

Crossings                                     \
Bridge    Brooklyn Bridge Manhattan Bridge Queensboro Bridge   
Day                                                            
Friday            82642.0         145166.0          118076.0   
Monday            69020.0         123613.0           98119.0   
Saturday          62783.0         106309.0           99974.0   
Sunday            55426.0          92666.0           81340.0   
Thursday          70413.0         129458.0          105294.0   
Tuesday           55713.0         106862.0           86800.0   
Wednesday         80626.0         146328.0          114499.0   

                                Precipitation                   \
Bridge    Williamsburg Bridge Brooklyn Bridge Manhattan Bridge   
Day                                                              
Friday               176330.0          0.0160           0.0160   
Monday               140357.0          0.1200           0.1200   
Saturday             141589.0          0.0800           0.0800   
Sunday               112203.0          0.0225           0.0225   
Thursday             154518.0          0.0225           0.0225   
Tuesday              127652.0          0.1100           0.1100   
Wednesday            168469.0          0.0000           0.0000   

                                                 
Bridge    Queensboro Bridge Williamsburg Bridge  
Day                                              
Friday               0.0160              0.0160  
Monday               0.1200              0.1200  
Saturday             0.0800              0.0800  
Sunday               0.0225              0.0225  
Thursday             0.0225              0.0225  
Tuesday              0.1100              0.1100  
Wednesday            0.0000              0.0000

In [62]:
bikes.pivot_table(index=['Day', 'Bridge'],
                   aggfunc={'Crossings':'sum',
                             'Precipitation':'mean'})

Crossings  Precipitation
Day       Bridge                                       
Friday    Brooklyn Bridge        82642.0         0.0160
          Manhattan Bridge      145166.0         0.0160
          Queensboro Bridge     118076.0         0.0160
          Williamsburg Bridge   176330.0         0.0160
Monday    Brooklyn Bridge        69020.0         0.1200
          Manhattan Bridge      123613.0         0.1200
          Queensboro Bridge      98119.0         0.1200
          Williamsburg Bridge   140357.0         0.1200
Saturday  Brooklyn Bridge        62783.0         0.0800
          Manhattan Bridge      106309.0         0.0800
          Queensboro Bridge      99974.0         0.0800
          Williamsburg Bridge   141589.0         0.0800
Sunday    Brooklyn Bridge        55426.0         0.0225
          Manhattan Bridge       92666.0         0.0225
          Queensboro Bridge      81340.0         0.0225
          Williamsburg Bridge   112203.0         0.0225
Thursday  Brooklyn Bridge        70413.0         0.0225
          Manhattan Bridge      129458.0         0.0225
          Queensboro Bridge     105294.0         0.0225
          Williamsburg Bridge   154518.0         0.0225
Tuesday   Brooklyn Bridge        55713.0         0.1100
          Manhattan Bridge      106862.0         0.1100
          Queensboro Bridge      86800.0         0.1100
          Williamsburg Bridge   127652.0         0.1100
Wednesday Brooklyn Bridge        80626.0         0.0000
          Manhattan Bridge      146328.0         0.0000
          Queensboro Bridge     114499.0         0.0000
          Williamsburg Bridge   168469.0         0.0000